In [1]:
import os
import pickle
import uuid

import pandas as pd
import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

In [2]:
year = 2021
month = 3

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/green_tripdata_{year:04d}-{month:02d}.parquet'

In [3]:
# MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = "05eb31b5ce63488bb169e5c00f50e241"

# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

logged_model = f'../web-service-mlflow/mlartifacts/1/{RUN_ID}/artifacts/model'
# logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(logged_model)

In [4]:
def read_dataframe(filename: str):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
def apply_model(input_file, model, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    y_pred = model.predict(dicts)

    df['ride_id'] = [str(uuid.uuid4()) for i in range(len(df))]

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = RUN_ID
    
    df_result.to_parquet(output_file, index=False)

In [7]:
apply_model(input_file, model, output_file)

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff
0,4bfc3ee4-d594-4582-8a79-f1d757a4c6b2,2021-01-01 00:15:56,43,151,3.933333,6.862711,-2.929378
1,09f9c6ee-ccf9-4957-8421-d5ac0f40ae36,2021-01-01 00:25:59,166,239,8.750000,13.368721,-4.618721
2,d31aa220-22f9-4e35-a757-1b92f8ca61bc,2021-01-01 00:45:57,41,42,5.966667,6.360871,-0.394204
3,1d7d5cb1-77ba-42c8-b4f8-cc98ba3c6f4f,2020-12-31 23:57:51,168,75,7.083333,11.824423,-4.741089
7,04921278-d467-439e-a137-a69d13158f50,2021-01-01 00:26:31,75,75,2.316667,3.389290,-1.072623
...,...,...,...,...,...,...,...
76513,26ab31ac-6333-441f-b091-89d66e3a49ee,2021-01-31 21:38:00,81,90,38.000000,41.526829,-3.526829
76514,8b566031-e3c2-45a0-ae0f-78502c975183,2021-01-31 22:43:00,35,213,38.000000,43.858974,-5.858974
76515,3ee6d146-9061-4b21-b091-1af4593ccc4d,2021-01-31 22:16:00,74,69,11.000000,14.436509,-3.436509
76516,ef601773-3f2a-4d1d-801b-353bdd836b0b,2021-01-31 23:10:00,168,215,27.000000,37.092622,-10.092622
